<a href="https://colab.research.google.com/github/gerardo0303/CSC-380-Update-BestBuy-Data/blob/main/Updateing_BB_files.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this Project I will be creating a pandas dataframe from nested XML file with an old BestBuy dataset from a public Amazon S3 bucket. Everthing that I do in this example project comes form multiple sorces to add new collemes to so the level of catagory and product is.

1.How to create Pandas DataFrame from nested XML? (https://www.geeksforgeeks.org/how-to-create-pandas-dataframe-from-nested-xml/)

2."Converting multi layered xml files to dataframes in python using xmltree" (https://towardsdatascience.com/converting-multi-layered-xml-files-to-dataframes-in-python-using-xmltree-a13f9b043b48"

3."How to read nested xml file with python pandas?" (https://stackoverflow.com/questions/71361593/how-to-read-nested-xml-file-with-python-pandas)

4."xml.etree.ElementTree — The ElementTree XML API" (https://docs.python.org/3/library/xml.etree.elementtree.html)

5."pandas.DataFrame.merge" (https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.merge.html#)

Let Begin with importing and unzipping the BestBuy dataset. Then we will install "lxml" a XML toolkit from the python pip package library.

In [ ]:
!curl https://ecommwebsitedata.s3.amazonaws.com/acm-sf-chapter-hackathon-big.zip --output BB_data.zip # all this is taken from the BestBuyPlayground.ipynb google colb notebook
!ls
!unzip BB_data.zip
!tar xvf product_data.tar.gz
!pip install lxml

We now import the modules needed to create the project.

In [13]:
import pandas as pd
import xml.etree.ElementTree as ETree

I then create a function to define the XML file path from the dataset

In [14]:
def Categories_File():
    return "product_data/categories/categories_0001_abcat0010000_to_pcmcat99300050000.xml"

Using pandas we are able to use pd.read_xml to read the xml file. Specifically the categories file.

In [15]:
read_catagories = pd.read_xml(Categories_File())
tree = ETree.parse(Categories_File())
root = tree.getroot()
#print (root)
all_category_data = [] #list of all the data

To be able to read the actaul path of the data. I found that using an element tree would be a simple way to do it. This would hlep keep track of how many catagoires it took to get to that item. Creating an element tree would allow is to create a path where we then can use it create the different levels of the categories.

In [16]:
for category_item in root.iter("category"): #loop through every possible path inside the tree while also looking at the levels of the item
    #if the loop finds the catagory tag with a "path"
    if len(category_item) > 2: #Every item has an id number and name with a path and if it doesn't have a path it would be less than 2
        category_data = []
        category_id = category_item.find("id").text #getting all the "id" from the catagories
        category_name = category_item.find("name").text #getting all the "name" from the catagories
        level_of_category = [None for i in range(5)] #makes a list with a length of 5 with nothing in it then will fill with category levels presented through path.
        #the level of category starts at 0 and gets greater as catagory levels are presented through it's path. For every item that has a path with levels we will look at the "id" to place it in a level. 
        i = 0 
        for path in category_item[2]: 
            level_of_category[i] = path.find("id").text 
            if i < 4:
                i += 1
                
        category_data = [category_name, category_id] + level_of_category
        all_category_data.append(category_data)


making the xml file to the dataframe and adding the catagories. 

In [17]:
xmlToDf = pd.DataFrame(all_category_data, columns=['name', 'category', 'category_l1', 'category_l2', 'category_l3', 'category_l4', 'category_l5'])
xmlToDf.head()

,name,category,category_l1,category_l2,category_l3,category_l4,category_l5
0,Gift Center,abcat0010000,cat00000,abcat0010000,None,None,None
1,Her,abcat0011000,cat00000,abcat0010000,abcat0011000,None,None
2,Leisure Gifts,abcat0011001,cat00000,abcat0010000,abcat0011000,abcat0011001,None
3,Kitchen Essentials,abcat0011002,cat00000,abcat0010000,abcat0011000,abcat0011002,None
4,Electronics,abcat0011003,cat00000,abcat0010000,abcat0011000,abcat0011003,None


Using "pd.read_csv" to reading/scan the train and tests file to be able update as well as add the levels of the categories. 

In [18]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

We then merge the levels created to the original train file copy

In [19]:
train_file_update = pd.merge(train, xmlToDf, on='category')
train_file_update.head() #printing a table of the file being merged

,user,sku,category,query,click_time,query_time,name,category_l1,category_l2,category_l3,category_l4,category_l5
0,000000df17cd56a5df4a94074e133c9d4739fae3,2125233,abcat0101001,Televisiones Panasonic 50 pulgadas,2011-09-01 23:44:52.533,2011-09-01 23:43:59.752,All Flat-Panel TVs,cat00000,abcat0100000,abcat0101000,abcat0101001,None
1,000001928162247ffaf63185cd8b2a244c78e7c6,2009324,abcat0101001,Sharp,2011-09-05 12:25:37.42,2011-09-05 12:25:01.187,All Flat-Panel TVs,cat00000,abcat0100000,abcat0101000,abcat0101001,None
2,000017f79c2b5da56721f22f9fdd726b13daf8e8,2877125,abcat0101001,rca,2011-10-25 07:18:14.722,2011-10-25 07:16:51.759,All Flat-Panel TVs,cat00000,abcat0100000,abcat0101000,abcat0101001,None
3,0000c4e9d7075985d1020c456e7ce36f83f834eb,2126065,abcat0101001,Samsung 40,2011-09-28 17:27:11.184,2011-09-28 17:26:15.832,All Flat-Panel TVs,cat00000,abcat0100000,abcat0101000,abcat0101001,None
4,00017f7beeac02736c0ce7bf1e75f3010939b34e,1831054,abcat0101001,lcd tv,2011-09-28 07:26:32.153,2011-09-28 07:26:21.476,All Flat-Panel TVs,cat00000,abcat0100000,abcat0101000,abcat0101001,None


We then also merge the levels created to the original test file copy

In [20]:
test_file_update = pd.merge(test, xmlToDf, on='category')
test_file_update.head(103) #checking the result of 103 prodects

,user,category,query,click_time,query_time,name,category_l1,category_l2,category_l3,category_l4,category_l5
0,000000df17cd56a5df4a94074e133c9d4739fae3,pcmcat143200050016,Televisiones,2011-09-01 23:43:01.633,2011-09-01 23:42:31.317,Televisions,cat00000,pcmcat139900050002,pcmcat143200050016,None,None
1,0024c1a452798225e6234b9a23bd85aae89e3789,pcmcat143200050016,Un55c6500vf,2011-10-05 16:57:38.004,2011-10-05 16:57:24.874,Televisions,cat00000,pcmcat139900050002,pcmcat143200050016,None,None
2,00a0e6627d300bc9c78b1c46bd5ec8a121e8bab3,pcmcat143200050016,vizio television remote control,2011-10-02 13:04:01.225,2011-10-02 13:02:56.484,Televisions,cat00000,pcmcat139900050002,pcmcat143200050016,None,None
3,00e8a110359937e091af31b669c412412d2d1b4b,pcmcat143200050016,2371074 2126339 2126108,2011-10-22 19:59:06.918,2011-10-22 19:57:46.54,Televisions,cat00000,pcmcat139900050002,pcmcat143200050016,None,None
4,011ec1789027ef04db6ee9af2c8d3008ef088d7d,pcmcat143200050016,FallSale_GPS_Car_20110930,2011-09-30 23:29:08.986,2011-09-30 23:24:58.905,Televisions,cat00000,pcmcat139900050002,pcmcat143200050016,None,None
...,...,...,...,...,...,...,...,...,...,...,...
98,1f10c3a4d4a3dd87f4cf13e4629d479f42f8bbb0,pcmcat143200050016,2622037 2127204 2127213 2121716 2138291,2011-10-08 15:24:09.669,2011-10-08 15:22:36.468,Televisions,cat00000,pcmcat139900050002,pcmcat143200050016,None,None
99,1f7ef69223d4f3812ee6ac94806a667e9ba1618c,pcmcat143200050016,Dynex,2011-10-03 23:26:01.306,2011-10-03 23:24:18.547,Televisions,cat00000,pcmcat139900050002,pcmcat143200050016,None,None
100,1f83530b8f5d94fc97e976ab702d73699b2fd274,pcmcat143200050016,2622037 2127204 2127213 2121716 2138291,2011-10-05 21:51:30.173,2011-10-05 21:50:02.511,Televisions,cat00000,pcmcat139900050002,pcmcat143200050016,None,None
101,1fd56a9e612330b55aed5c95b5e9f08ab01becf0,pcmcat143200050016,televisions,2011-09-21 22:49:34.132,2011-09-21 22:49:22.936,Televisions,cat00000,pcmcat139900050002,pcmcat143200050016,None,None


Finally we update the original train and test file to new ones called "New_train_File.csv" and "New_test_File.csv"

In [21]:
pd.merge(train, xmlToDf, on='category').to_csv('New_train_File.csv')
pd.merge(test, xmlToDf, on='category').to_csv('New_test_File.csv')